In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.plotting import scatter_matrix
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import mutual_info_classif
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import warnings  
warnings.filterwarnings('ignore')

### Heart Attack EDA and ML model building
    First time analyzing and building ML model in kaggle
    Hope i can improve over time
    Any critics helps

In [ ]:
heart_attack = pd.read_csv('../input/heart-attack-analysis-prediction-dataset/heart.csv')
heart_attack

In [ ]:
#check for null values
heart_attack.isnull().sum()
#no null values

In [ ]:
#check dtypes
heart_attack.dtypes
#all numeric type
#but some features must be categorical, so the data is cleaned and changed to numeric
#must find which features are categorical


### so since the data is already cleaned, we can just start the EDA
#### first analze each features and later, their coorelation with our target

#### 1. Age : 
    does certain age range has higher heart attack risk? 
    from our dataset, visualize 0 and 1 (target) and compare the age range

#### 2. Sex (0=Female 1=Male) :
    Does one gender has higer risk of heart attack over ther other?
    

#### EXNG (exercise induce angina) : 
    angina is chest pain or pressure, usually due to insufficient blood flow to the heart muscle
    in the dataset value 1 if angina is caused by exercise and 0 if caused by other reason 

#### CA: number of major vessels
        Does low or high number has a correlation to higher heart attack risk?
        
#### Chest Pain : the most important feature i think   
    Value 1: typical angina
    Value 2: atypical angina
    Value 3: non-anginal pain
    Value 4: asymptomatic
    
    Visualize between 0 and 1, and put hue = cp, so we can see in comparison
     in higher chance of heart attack (1 in our target), which chest pain value is most dominan
     in lower chance (0 in our target) which cp value is more dominan
     in here we can see which value is more likely linked to higher heart attack risk

#### trtbps : resting blood pressure (in mm Hg)
    usually resting blood pressure is denoted as systolic/diatolic (e.g. 120/80 Mmhg)
    but in our dataset it seems only the systolic part is recorded
    normal trtbps for nomal perso is  below 120/80 mm Hg and above 90/60 mm Hg in an adult. 
    does higher trtbps result in higher risk oh Heart attack?
    visualize the target and trtbps and AGE, usually older people tend to have higher trtbps.
        
    "As you age, the vascular system changes. This includes your heart and blood vessels. 
    In the blood vessels, there’s a reduction in elastic tissue in your arteries, 
    causing them to become stiffer and less compliant. As a result, your blood pressure increases". 
    as describe from nakano in healthline.com 
        
    we have to find if the high risk group is dominantly occupied by old people (above 40) and have high blood pressure, 
    visualize this for easier initial analysis
        
#### chol : cholestoral in mg/dl fetched via BMI sensor

    it is well known that High cholesterol can limit blood flow, increasing the risk of a heart attack or stroke. 
    Total cholesterol levels less than 200 milligrams per deciliter (mg/dL) are considered desirable for adults.
    A reading between 200 and 239 mg/dL is considered borderline high and a reading of 240 mg/dL and above is considered 
    high
    
    need to confirm if high chol results in higher heart attack risk
    again, visualize the target with our chol feature, in high risk group what is the most common range of cholestrol
    also, cholestrol has higher chance of being high for old people
 
#### fbs : (fasting blood sugar > 120 mg/dl) (1 = true; 0 = false)
    fasting blood sugar is the level of blood sugar taken after a patient has fast for certain amount period of time. 
    Does high blood sugar level (more than 120 mg/dl) has higher chance for a heart attack?

#### rest_ecg : resting electrocardiographic results
    Value 0: normal
    Value 1: having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV)
    Value 2: showing probable or definite left ventricular hypertrophy by Estes' criteria
    
    resting ecg taken while the patient rest (there is also ecg taken while the patient is running on treadmill)
    
    does higher rest_ecg result (0, 1, 2) has higher chance of heart attack?
    
#### thalach : maximum heart rate achieved
      does higher thalach result in higher risk?
       
#### thall - 3 = normal; 6 = fixed defect; 7 = reversable defect

#### oldpeak - ST depression induced by exercise relative to rest
#### slope - the slope of the peak exercise ST segment (1 = upsloping; 2 = flat; 3 = downsloping)

#### Target (0 = Low Heart attack risk, 1=high attack risk)

In [ ]:
#Since the data is already cleaned and no null-values, we can go straight to EDA and build predictive model later
#we would be able to know which feature is categorical and numerical through  features explanation
#but we can also make sure of it by showing each features cardinality
#categorical data will have low cardinality, and doing this will also helps us in building predictive model later

heart_attack.nunique()


#there are 303 rows, and these are health informations of each sample

#age : numerical
#sex : categorical
#cp (chest pain) : categorical (1: typical angina,  2: atypical angina,  3: non-anginal pain,  4: asymptomatic)
#trtbps (resting blood pressure) : numerical
# chol (cholesterol) : numerical
# fbs (fasting blood sugar) : categorical ((fasting blood sugar > 120 mg/dl) (1 = true; 0 = false))
# restecg (resting electrocardiographic results) :  categorical 
#thalachh (maximum heart rate achieve) : numerical
#exng (exercise induce angina) : categorical
#oldpeak (ST depression induced by exercise relative to rest) : numerical
#slp (slope, slope of the peak exercise ST segment (1 = upsloping; 2 = flat; 3 = downsloping)) : categorical
#caa ( number of major vessels): categorical
#thal : categorical


#output is the target (0: low heart attack risk, 1: high heart attack risk)

In [ ]:
#classify the features 

numerical_features = ['age', 'trtbps', 'chol', 'thalachh', 'oldpeak']

categorical_features = ['sex', 'cp', 'fbs', 'restecg', 'exng', 'slp', 'caa', 'thall']

target = heart_attack['output']

heart_attack =  heart_attack.drop(['output'], axis=1)


## Univariate Analysis

### Numerical Data

In [ ]:
# Descriptive statistic
# numerical features only
heart_attack[numerical_features].describe()

### age
    mean of age is 54
    minimum age is 29, so no children, teen or early adult data
    maximum age is 77 years old
    median is 55 years old, close to our mean, there are probably no outliers, 
        but we still have to check for outliers with histogram or distribution plot later 
        standard deviation is 9, it means if our dataset distribution is normal, 68% of age values wil be around 45-63 
        years old and 95% of age values will be around 36-72years old

#### resting blood pressure
    it seems only systolic is recorded
    mean and median is close to each other 131 and 130, so probably no outliers
        this considered as high value because normal people trtbps is below 120/80 Mmhg and above 90/60 Mmhg
        but we have to check the correlation with age, because older people then to have higher trtbs
    minimum trtbs value is 94, the minimum value recorded in dataset  (healthy value)
    maximum value is 200, it's hypertension, and people who have this high blood pressure
        will experience chest pain, headache, shortness of breath or blood in the urine
        it is the the highest trtbps in our dataset
    standard deviation is 17, with our means of 131 (assuming no outliers and bell-shaped distribution),
        68% of trtbps values will be around 114-148
      
#### cholesterol
    mean and median is still pretty close at 241 and 246 , but there maybe small amounts of outliers
    these numbers are not good numbers
    Total cholesterol levels less than 200 milligrams per deciliter (mg/dL) are considered desirable for adults. 
    A reading between 200 and 239 mg/dL is considered borderline high
    and a reading of 240 mg/dL and above is considered high
    minimum and maximum values are 126 and 564, 
        126 is borderline low (too low can lead to another type of health risk, but we wouldn't explain it further)
        564 is too high  (this might be sign of obesity , and can cause symptoms; angina and chest pain)
    standard deviation is 51, assuming no outliers (removing or transform) and bell-shape distribution,
        68% of chol values is around 190-297
    
    
####  thalachh (maximum heart rate achieved)
     mean and median 149-153, there maybe small amount of outliers
         according to heart.org, good heart rate per person is vary depend on age (220-age)
     standard deviation 22 (68% of data will be around 127-175)
     min and max values are 71 and 202

#### oldpeak (ST depression induced by exercise relative to rest)
    still have not idea what this feature means, but it is in our dataset
    so this must be important value to determine the risk ofheart attack
    we can understand this further later in bivariate analysis and features importance
        

In [ ]:
#histogram to see our distribution
heart_attack[numerical_features].hist(bins=30, figsize= (10, 7))
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
#for i in list(heart_attack[numerical_features].columns):
heart_attack[numerical_features].boxplot()
#ax.set_xticklabels(list(heart_attack[numerical_features].columns))
plt.show()

#### Distribution Analysis
    Age and Thallach seems to follow normal distribution
    Cholesterol has outliers and skewed to the right
    Trtbps has few outliers
    Oldpeak is highly skewed to the right with long tail
    
    Since i will use KNN classifier later, it is necessary to transform these numerical features

### Categorical Data

In [ ]:
#fig, ax = plt.subplots(figsize=(10, 8))

translate_output = {'output' : {1: 'High Heart Attack Risk',  0 : 'Low Heart Attack Risk'}}

# Pie chart
labels = list(target.value_counts().reset_index()['index'])
labels = [translate_output['output'][i] for i in labels]
sizes = list(target.value_counts().reset_index()['output'])

#colors
colors = ['#ff9999','#66b3ff']
#explsion
explode = (0.05,0.05)
fig, ax = plt.subplots(figsize=(8, 5)) 
ax.pie(sizes, colors = colors, labels=labels, autopct='%1.1f%%', startangle=90, pctdistance=0.85, explode = explode)
#draw circle
centre_circle = plt.Circle((0,0),0.70,fc='white')
fig = plt.gcf()
fig.gca().add_artist(centre_circle)
# Equal aspect ratio ensures that pie is drawn as a circle
ax.axis('equal')  
plt.tight_layout()
plt.show()

   54.5% sample in the dataset has high chance of heart attack

In [ ]:
#Categorical features 
#starts with this question, what is the dominant value of each cat features?


translate_values = {'sex' : {0: 'Female', 1:'Male'},
                   'cp' : {0: 'typical angina', 1: 'atypical angina', 2: 'non-anginal pain', 3: 'asymptomatic'}, 
                   'fbs': {0: 'Low FBS (<120 mg/dl)', 1: 'High FBS (>120 mg/dl)'}, 
                   'restecg': {0 : 'Normal', 1 : 'ST-T wave abnormality', 2: 'left ventricular hypertrophy'}, 
                   'exng' : {0: 'Other Angina Reason', 1 : 'Exercise Induce Angina'}, 
                   'slp' : {0: 'upsloping', 1 : 'flat', 2: 'downsloping'}, 
                   'caa' : {0:1, 1:2, 2:3, 3:4, 4:5},
                   'thall': {0:0, 1:1, 2:2, 3:3}}



fig = plt.figure(figsize=(15, 20))
for x in range(len(list(heart_attack[categorical_features].columns))):
    labels = list(heart_attack[categorical_features][categorical_features[x]].value_counts().reset_index()['index'])
    translated_label = [translate_values[categorical_features[x]][i] for i in labels] #translate label for readability
    sizes = [i for i in list(heart_attack[categorical_features][categorical_features[x]].value_counts().reset_index()[categorical_features[x]])]
    size_true = [int(i/sum(sizes)*100) for i in sizes]
    explode = tuple([0.05 for i in range(len(labels))])
    ax = plt.subplot(4, 2, x+1)
    ax.pie(size_true, labels=translated_label, autopct='%1.1f%%', startangle=90, pctdistance=0.85 , explode = explode) #, textprops={'fontsize': 20}
    centre_circle = plt.Circle((0,0),0.70,fc='white')
    centre_circle = plt.Circle((0,0),0.70,fc='white')
    fig = plt.gcf()
    fig.gca().add_artist(centre_circle)
    ax.axis('equal') 
    plt.title(categorical_features[x], fontsize=30)
    plt.tight_layout()
#fig.suptitle(year[x], fontsize=16, y=1.03)
plt.show()

### All analysis is based on dataset
    1. Sex : There are more Male than Female in our dataset
    2. Chest pain : 48% chest pain is caused by typical angina 
    3. Fbs : 85.9% sample has normal blood sugar level (<120mg/dl)
    4. Restecg : 48.5% has normal heartwave, 50.5 has ST-T wave abnormality
    5. Exng : 67.7% has Angina caused by other reasons than exercise

## Bivariate Analysis

### Numerical Features

In [ ]:
#try to use pairplot
fig1 = plt.figure(figsize=(16,16))
sns.pairplot(pd.concat([heart_attack[numerical_features], target], axis=1), hue='output')
plt.show()

### Analysis
    fromm scatterplot above, we only see  positive correlation between age and cholesterol, looks like older people tend to have higher cholesterol

In [ ]:
#heatmap
fig, ax = plt.subplots(figsize=(15,10))
sns.heatmap(pd.concat([heart_attack[numerical_features], target], axis=1).corr(),annot=True,cmap="Oranges",ax=ax)
plt.show()

#### No strong correlation
    Cp, thalachh, and slp show positive correlation but close to 0 rather than 1

### Categorical 

In [ ]:
#try to use pairplot
fig1 = plt.figure(figsize=(16,16))
sns.pairplot(pd.concat([heart_attack[categorical_features], target], axis=1), hue='output')
plt.show()


### Building Machine Learning Model

### Mutual Information Score

In [ ]:
def mi_scores(dataset, target):
    mutual_class = mutual_info_classif(dataset, target, random_state=42)
    mutual_class = pd.Series(mutual_class, name="mutual information scores", index=dataset.columns)
    mutual_class = mutual_class.sort_values(ascending=False)
    return mutual_class

mutual_info_score = mi_scores(heart_attack, target)
mutual_info_score # show a few features with their MI scores

In [ ]:
plt.figure(dpi=100, figsize=(8, 5))
mutual_info_score = mutual_info_score.sort_values(ascending=True)
width = np.arange(len(mutual_info_score))
ticks = list(mutual_info_score.index)
plt.barh(width, mutual_info_score, color= '#ff9999')
plt.yticks(width, ticks)
plt.title("Mutual Information Scores")


#### Mutual information analysis
    Thall, caa and cp have the strongest relationship with our target
    fbs, trtbps, and age has low relationship with our target
    meanwhile sex is independen and doesn't show any relationship with our target

### Normalize and Transform Features

In [ ]:
#before we go any further, 
#i want to Standardize and transform our numerical feature so it will be more gaussian like

robust_scale = RobustScaler()
power = PowerTransformer()
data_trans = robust_scale.fit_transform(heart_attack[numerical_features])
data_trans = power.fit_transform(data_trans)
df = pd.DataFrame(data_trans, columns=heart_attack[numerical_features].columns)
ax = df.hist(figsize=(10, 7))
plt.show()

In [ ]:
#replace old value with new value
heart_attack2 =heart_attack.copy()
heart_attack2.drop(labels=numerical_features, axis=1, inplace=True)
heart_attack2[numerical_features] = df[numerical_features]
heart_attack2

In [ ]:
#perform ONE-Hot Encoding for our Categorical feature
#all our categorical features has cardinality less than 5, so OHE would be ideal
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
data_trans2 = pd.DataFrame(OH_encoder.fit_transform(heart_attack2[categorical_features]))
data_trans2.index = heart_attack2.index
heart_attack2 = heart_attack2.drop(categorical_features, axis=1)
heart_attack2 = pd.concat([heart_attack2, data_trans2], axis=1)
#now our dataset is ready for any machine learning model
heart_attack2

In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(heart_attack2, target, train_size=0.8, test_size=0.2, random_state=0)

### KNN

In [ ]:
#KNN
knn = KNeighborsClassifier(n_neighbors=10)

# Train the model using the training sets
knn.fit(x_train,y_train)

#Predict Output
y_pred = knn.predict(x_valid)
knn_score = metrics.accuracy_score(y_valid, y_pred)
print("Accuracy:", knn_score)

### Random Forest Classifier

In [ ]:
# creating Random Forest Classifier Object
rfc = RandomForestClassifier(n_estimators = 100, random_state=42)  
  
# Training the model 
rfc.fit(x_train, y_train)
  
#prediction using validation set
y_pred = rfc.predict(x_valid)
  
rfc_score = metrics.accuracy_score(y_valid, y_pred)
  
# accuracy
print("ACCURACY OF THE MODEL: ", rfc_score)

### Score Summary KNN VS Random Forest Classifier

In [ ]:
score = pd.DataFrame({'KNN': [knn_score], 'Random Forest Classifier' : [rfc_score]})
score